In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style("white")
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import chisquare, chi2_contingency, shapiro

In [2]:
# Load preprocessed data - remove top and bottom 15% rows
data = pd.read_csv('Data/processed_data_final.csv')
# data_org = pd.read_csv('Data/data_original_final.csv')

In [3]:
# data_org.head()
var_list = ['Vti','Vte', 'Inspiratory_time', 'Expiratory_time', 'leak_diff_per']
data[var_list].describe()
# .to_csv('EDA.csv')

,Vti,Vte,Inspiratory_time,Expiratory_time,leak_diff_per
count,13356.000000,13356.000000,13356.000000,13356.000000,13356.000000
mean,19.518065,10.113956,534.556080,497.700135,32.723649
std,14.556603,5.052690,282.761902,160.335249,26.230742
min,2.130000,0.060000,93.000000,0.000000,0.000000
25%,6.830000,5.800000,300.000000,387.000000,3.069780
50%,18.600000,7.690000,561.000000,465.000000,40.969509
75%,29.680000,15.350000,741.000000,549.000000,50.598657
max,98.530000,28.550000,1986.000000,1500.000000,99.490178


In [8]:
data.groupby('File_No').count()

,Feedback,Time,Breath,Vti,Vte,Inspiratory_time,Expiratory_time,Participant,feedback,technique,mannikin,role,years,NLS,leak_diff,leak_diff_per,ins_exp,inflation
File_No,,,,,,,,,,,,,,,,,,
0_NF-73701-0023,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
100_NF-73701-0023,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
101_F-73701-0023,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37
102_F-73701-0023,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48
103_F-73701-0023,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96_F-73701-0023,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50
97_NF-73701-0023,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29
98_NF-73701-0023,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45


In [17]:
data.groupby('File_No').count()['Inspiratory_time'].median()

45.0

In [36]:
# Create conditions
var_list = ['leak_20', 'insp_500', 'Vti_40']
condition_list =['leak_diff_per', 'Inspiratory_time', 'Vti']
i=0
var=var_list[i]
# print(var)


# Condition leak_30
# ['mannikin','role', 'Feedback']
# con1 =data.groupby(['mannikin', 'technique','role','Feedback'])[condition_list[0]].apply(lambda x: (x < 30).sum()).reset_index(name='Breaths_<30')
# con2 = data.groupby(['mannikin', 'technique','role','Feedback'])[condition_list[0]].apply(lambda x: ((x >= 30)& (x <=60) ).sum()).reset_index(name='Breaths_30-60')
# con3= data.groupby(['mannikin', 'technique','role','Feedback'])[condition_list[0]].apply(lambda x: (x > 60).sum()).reset_index(name='Breaths_>60')

# con1 =data.groupby('Feedback')[condition_list[0]].apply(lambda x: (x < 30).sum()).reset_index(name='Breaths_<30')
# con2 = data.groupby('Feedback')[condition_list[0]].apply(lambda x: ((x >= 30)& (x <=60) ).sum()).reset_index(name='Breaths_30-60')
# con3= data.groupby('Feedback')[condition_list[0]].apply(lambda x: (x > 60).sum()).reset_index(name='Breaths_>60')

# con1 =data.groupby(['mannikin','Feedback'])[condition_list[i]].apply(lambda x: (x < 30).sum()).reset_index(name='Breaths_<30')
# con2 = data.groupby(['mannikin','Feedback'])[condition_list[i]].apply(lambda x: ((x >= 30)& (x <=60) ).sum()).reset_index(name='Breaths_30-60')
# con3= data.groupby(['mannikin', 'Feedback'])[condition_list[i]].apply(lambda x: (x > 60).sum()).reset_index(name='Breaths_>60')

# # Condition ventilatory rate
# con1 =data.groupby(['mannikin', 'technique','role','Feedback'])[condition_list[1]].apply(lambda x: (x < 500).sum()).reset_index(name='Ventilarory>60')
# con2 = data.groupby(['mannikin', 'technique','role','Feedback'])[condition_list[1]].apply(lambda x: ((x >= 500)& (x <=750) ).sum()).reset_index(name='Ven_40_60')
# con3= data.groupby(['mannikin', 'technique','role','Feedback'])[condition_list[1]].apply(lambda x: (x > 750).sum()).reset_index(name='Ventilarory<40')

# Condition Vti
# print(var_list[2])
con1 =data.groupby(['mannikin', 'technique','role','Feedback'])[condition_list[2]].apply(lambda x: (x < 20).sum()).reset_index(name='Vti<20')
con2 = data.groupby(['mannikin', 'technique','role','Feedback'])[condition_list[2]].apply(lambda x: ((x >= 20)& (x <=40) ).sum()).reset_index(name='Vti_20-40')
con3= data.groupby(['mannikin', 'technique','role','Feedback'])[condition_list[2]].apply(lambda x: (x > 40).sum()).reset_index(name='Vti>40')

In [37]:
df_count = pd.concat([con1,con2, con3], axis=1)
df_count = df_count.loc[:, ~df_count.columns.duplicated()]
df_count.to_csv('chi_Vi_term.csv', index=False)
df_count

,mannikin,technique,role,Feedback,Vti<20,Vti_20-40,Vti>40
0,1,1,1,F,590,2,1
1,1,1,1,NF,542,1,1
2,1,1,2,F,527,3,0
3,1,1,2,NF,361,51,6
4,1,1,3,F,453,9,4
5,1,1,3,NF,298,19,10
6,1,2,1,F,674,4,0
7,1,2,1,NF,644,1,1
8,1,2,2,F,640,0,0
9,1,2,2,NF,597,13,0


In [44]:
df_chi = df_count.query('technique==2 & mannikin==2 & role==3')
drop_list = ['technique', 'mannikin', 'role']
df_chi = df_chi.drop(drop_list, axis=1)
# df_chi = df_count
df_chi.set_index('Feedback', inplace=True)

# Chi-square test
stat, p, dof, expected = chi2_contingency(df_chi)
print('The Chi-Square value is {:.3f}.\nDegree of freedom is {}.\nP-value is {}'.format(stat,dof,p))
print('Contingency table:', expected)
print(df_chi.sum(axis=1))


The Chi-Square value is 159.398.
Degree of freedom is 2.
P-value is 2.4391681394144734e-35
Contingency table: [[  0.4952627  443.75538329 130.74935401]
 [  0.5047373  452.24461671 133.25064599]]
Feedback
F     575
NF    586
dtype: int64
